In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

sys.path.append('..')
from genotype_evaluator import GenotypeEvaluator

%matplotlib inline

## Umda implementation

In [11]:
def random_individual(probs):
    x = np.empty(probs.shape[0], dtype=np.int32)
    for i in range(probs.shape[0]):
        x[i] = np.random.choice(3, p=[probs[i][0], probs[i][1], probs[i][2]])
    return x


def random_population(probs, population_size):
    population = np.empty((population_size, probs.shape[0]), dtype=np.int32)
    for i in range(population_size):
        population[i] = random_individual(probs)
    return population


def select_samples_indexes(objective_function, samples, best_samples_size):
    EPSILON_PROBABILITY = 1e-4
    adaptations = objective_function(samples)
    adaptations = adaptations - np.min(adaptations)
    if np.sum(adaptations) == 0:
        adaptations = np.repeat(1.0, samples.shape[0])
    adaptations[adaptations == 0] = EPSILON_PROBABILITY
    adaptations = adaptations / np.sum(adaptations)
    return np.random.choice(samples.shape[0], best_samples_size, p=adaptations, replace=False)


def umda_model_estimation(samples):
    return np.vstack([
        np.average(samples == 0, axis=0),
        np.average(samples == 1, axis=0),
        np.average(samples == 2, axis=0)
    ]).transpose()
                
    
def umda(objective_function, dim_size, population_size, best_samples_size, max_time):
    p = np.ones((dim_size, 3)) / 3
    start_time = time.time()
    populations = []
    score_map = {}
    while time.time() - start_time < max_time:
        samples = random_population(p, population_size)
        populations.append(samples)
        score_values = objective_function(samples)
        print("Average score values: {}, best score value: {}".format(np.mean(score_values), np.max(score_values)))
        best_samples_indexes = select_samples_indexes(objective_function, samples, best_samples_size)
        p = umda_model_estimation(samples[best_samples_indexes])
    return populations, p

## Rounded board evaluator

In [7]:
genotype_evaluator = GenotypeEvaluator('../clusters/human100_cluster.c', ['../boards/round_board.b'])

In [ ]:
rb_populations, rb_probs = umda(genotype_evaluator.evaluate_genotypes, 100, 100, 20, 3600)

Average score values: 350.51, best score value: 679.0
Average score values: 370.44, best score value: 678.0
Average score values: 385.1, best score value: 679.0
Average score values: 399.57, best score value: 683.0
Average score values: 456.32, best score value: 986.0
Average score values: 491.14, best score value: 673.0
Average score values: 485.38, best score value: 688.0
Average score values: 495.77, best score value: 951.0
Average score values: 545.6, best score value: 2111.0
Average score values: 541.06, best score value: 2110.0
Average score values: 558.76, best score value: 1459.0
Average score values: 674.48, best score value: 2111.0
Average score values: 616.3, best score value: 2110.0
Average score values: 607.4, best score value: 2110.0
Average score values: 599.06, best score value: 1190.0
Average score values: 726.68, best score value: 2111.0
Average score values: 830.11, best score value: 2111.0
Average score values: 1088.39, best score value: 2111.0
Average score values:

In [ ]:
with open('../individuals/rb_umda_gen5.i', 'wb') as file_stream:
    best_index = np.argmax(genotype_evaluator.evaluate_genotypes(rb_populations[5]))
    gen5_individual = genotype_evaluator.individual_factory.create_individual(rb_populations[5][best_index])
    gen5_individual.marker_manager.game_state_marker_lines = {}
    pickle.dump(gen5_individual, file_stream)
with open('../individuals/rb_umda_best.i', 'wb') as file_stream:
    best_index = np.argmax(genotype_evaluator.evaluate_genotypes(rb_populations[-1]))
    best_individual = genotype_evaluator.individual_factory.create_individual(rb_populations[-1][best_index])
    best_individual.marker_manager.game_state_marker_lines = {}
    pickle.dump(best_individual, file_stream)